In [3]:
from transformers import pipeline
from transformers import GPT2Tokenizer
summarizer = pipeline('summarization')

In [4]:
import pickle
allCommentary = pickle.load(open('allCommentary', 'rb'))
allNews = pickle.load(open('allNews', 'rb'))

In [24]:
example = allCommentary['Processed'].iloc[6]

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
x = tokenizer.encode(example)


In [23]:
#Here is the processed text from one of the articles
allCommentary['Processed'].iloc[6]

'Chief Justice Rehnquists hospitalization for thyroid cancer treatment has provided a sudden and poignant reminder to many voters that the winning candidate in next Tuesdays election is likely to make US court appointments that could have profound political consequences. The future of the Supreme Court, which has barely registered with most voters, may now become an important campaign issue. Rehnquists illness is no more than a catalyst for activation of this issue since the advanced ages of the Justices made vacancies likely before Rehnquists illness, and there is no reason to suppose that Rehnquists resignation is imminent since Rehnquist is scheduled to return to the bench next week. It is impossible to predict which, if either, candidate will benefit from the renewal of interest in the Court, although a recent Time Magazine poll indicated that the prospect of Supreme Court appointments made voters slightly more inclined to vote for Kerry. At the very least, both parties are likely 

In [25]:
#Here is the summary
#This is pretty good!
print(summarizer(example, max_length=130, min_length=30))

[{'summary_text': 'Chief Justice Rehnquists hospitalization for thyroid cancer treatment has provided a sudden and poignant reminder to many voters that the winning candidate in next Tuesdays election is likely to make US court appointments. The future of the Supreme Court, which has barely registered with most voters, may now become an important campaign issue.'}]


Essentially, one future direction of this project could be to be able to summarize articles on a certain topic from each year, so you can see how updates on that country/topic changed year to year

In [ ]:
########IGNORE BELOW FOR NOW: JUST PLAYING AROUND WITH BART####################

In [41]:
#model_class, tokenizer_class, pretrained_weights = BertModel, BertTokenizer, 'bert-base-uncased'
#tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
#model = model_class.from_pretrained(pretrained_weights)

text = allCommentary['Processed'].iloc[0]
text = tokenizer.tokenize(text)
if len(text) > 512:
    text = text[:512]
#text = "After a morning of Thrift Store hunting, a friend and I were thinking of lunch"

#text = tokenizer.encode(text, add_special_tokens=True, max_length=10)
#print(text)
#print(len(text))
#print(summarizer(text, max_length=130, min_length=30))

Your min_length is set to 30, but you input_length is only 8. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 130, but you input_length is only 8. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


KeyboardInterrupt: 

In [9]:
text = allCommentary['Processed'].iloc[0]
text = text.split()
text = text[:1000]
print(summarizer(text, max_length=130, min_length=30))

Your min_length is set to 30, but you input_length is only 8. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 130, but you input_length is only 8. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


KeyboardInterrupt: 

In [27]:
source_text = allCommentary['Processed'].iloc[0]

input_ids = tokenizer.batch_encode_plus([source_text], return_tensors='pt', maxlength=1024)

summary_ids = model.generate(input_ids, num_beams=4, length_penalty=2.0,
                             max_length=142,min_length=5,no_repeat_ngram_size=3)
summary_text = tokenizer.decode(summary_ids.squeeze(),skip_special_tokens=True)


Token indices sequence length is longer than the specified maximum sequence length for this model (3733 > 1024). Running this sequence through the model will result in indexing errors


NameError: name 'model' is not defined

In [33]:
from transformers import BartForConditionalGeneration, BartTokenizer
from typing import List

def old_summarization_pipeline(text: List[str]) -> List[str]:
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
    model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
    input_ids = tokenizer.batch_encode_plus(text, return_tensors='pt', max_length=1024)['input_ids']
    summary_ids = model.generate(input_ids)
    summaries = [tokenizer.decode(s) for s in summary_ids]
    return summaries

In [37]:
old_summarization_pipeline(allCommentary['Processed'].iloc[0].split())

ValueError: The sequences building the batch are not of the same size, no tensor can be built. Set `pad_to_max_length=True` to pad the smaller sequencesup to the larger sequence's length.